In [7]:
import torch
import csv
import string
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(NeuralNetwork, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, (hn, cn) = self.lstm(x)
        output = self.fc(hn[-1])
        return output

# Funció per netejar el text
def netejar_text(text):
    text = text.lower()  # Convertir tot a minúscules
    text = ''.join([caracter for caracter in text if caracter.isalpha() or caracter == ' '])  
    # Conservar només lletres i espais
    return text

# Llegir i processar el CSV
def processar_csv(ruta_csv):
    ressenyes = []
    etiquetes = []

    with open(ruta_csv, mode='r', encoding='utf-8') as arxiu:
        lector_csv = csv.reader(arxiu)
        next(lector_csv)  # Saltar la primera línia (encapçalament)
        for fila in lector_csv:
            etiqueta = None
            if fila[0] == 'positive':
                etiqueta = 1
            elif fila[0] == 'negative':
                etiqueta = 0
            
            ressenya = netejar_text(fila[1])  # Neteja del text a la columna 1
            ressenyes.append(ressenya)
            etiquetes.append(etiqueta)
        
        return ressenyes, etiquetes


# Tokenitzar les ressenyes
def tokenitzar_ressenyes(ressenyes):
    paraula_a_index = {}
    ressenyes_tokenitzades = []
    index = 0

    for ressenya in ressenyes:
        tokens = ressenya.split()
        tokenitzada = []
        for paraula in tokens:
            if paraula not in paraula_a_index:
                paraula_a_index[paraula] = index
                index += 1
            tokenitzada.append(paraula_a_index[paraula])
        ressenyes_tokenitzades.append(tokenitzada)
    return ressenyes_tokenitzades, paraula_a_index

# Ruta de l'arxiu CSV
ruta_csv = "/home/itibcn/Desktop/Torch/datasets/IMDB/IMDBDataset.csv"

# Processar les dades del CSV
ressenyes, etiquetes = processar_csv(ruta_csv)
if ressenyes:
    ressenyes_tokenitzades, paraula_a_index = tokenitzar_ressenyes(ressenyes)
    print(f"Les 50 primeres ressenyes tokenitzades: {ressenyes_tokenitzades[:50]}")
else:
    print("No s'han pogut processar dades.")


Les 50 primeres ressenyes tokenitzades: [[0], [0], [0], [1], [0], [0], [0], [1], [1], [0], [1], [1], [1], [1], [0], [1], [0], [1], [0], [1], [0], [1], [0], [1], [1], [0], [0], [1], [1], [0], [0], [0], [1], [0], [1], [1], [1], [1], [0], [1], [1], [0], [1], [1], [0], [0], [1], [1], [0], [1]]


# LSTM

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import re

# --- Pas 1: Configuració i preprocessing ---
def netejar_text(text):
    # Convertir a minúscules, treure símbols especials
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Eliminar caràcters no alfanumèrics
    text = text.strip()  # Eliminar espais innecessaris
    return text


def veure_prediccions(model, dataloader, paraula_a_index):
    model.eval()
    idx_a_paraula = {index: paraula for paraula, index in paraula_a_index.items()} 
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            output = model(X_batch).to(device)
            pred = (output.squeeze() > 0.5).float()
            for seq, etiqueta, prediccio in zip(X_batch, y_batch, pred):
                # Convertir la seqüència d'índexs a text
                ressenya = " ".join([idx_a_paraula[idx.item()] for idx in seq if idx.item() in idx_a_paraula])
                etiqueta_real = "Positive" if etiqueta.item() == 1 else "Negative"
                prediccio_str = "Positive" if prediccio.item() == 1 else "Negative"
                print(f"Ressenya: {ressenya}")
                print(f"Etiqueta Real: {etiqueta_real}, Predicció: {prediccio_str}\n")
    
    
    # Llegir i processar el CSV
def processar_csv(ruta_csv):
    ressenyes = []
    etiquetes = []

    with open(ruta_csv, mode='r', encoding='utf-8') as arxiu:
        lector_csv = csv.reader(arxiu)
        next(lector_csv)  # Saltar la primera línia 

        for fila in lector_csv:
            if len(fila) < 2:
                # La fila no té suficients columnes
                print(f"Fila incorrecta: {fila}")
                continue
            
            etiqueta = None
            if fila[1] == 'positive':
                etiqueta = 1
            elif fila[1] == 'negative':
                etiqueta = 0

            # Comprovar si l'etiqueta és vàlida
            if etiqueta is None:
                print(f"Etiqueta no vàlida en fila: {fila}")
                continue

            ressenya = netejar_text(fila[1])
            if not ressenya.strip():
                print(f"Ressenya buida en fila: {fila}")
                continue

            ressenyes.append(ressenya)
            etiquetes.append(etiqueta)

    return ressenyes, etiquetes


# Potser s'hauria de cridar aquest mètode un cop per fora
def tokenitzar_ressenyes(ressenyes):
    """
    Convertim les ressenyes en tokens mitjançant un diccionari.
    """
    tokens = []
    paraula_a_index = {}
    index = 0
    for ressenya in ressenyes:
        seq = []
        for paraula in ressenya.split():
            if paraula not in paraula_a_index:
                paraula_a_index[paraula] = index
                index += 1
            seq.append(paraula_a_index[paraula])
        tokens.append(seq)
    return tokens, paraula_a_index

def padding_sequencies(sequencies, max_len):
    padded = np.zeros((len(sequencies), max_len), dtype=int)
    for i, seq in enumerate(sequencies):
        padded[i, :len(seq)] = seq[:max_len]
    return padded

# Processar les dades
max_len = 100  # Longitud màxima de les ressenyes (padding)
ressenyes, etiquetes = processar_csv(ruta_csv)
if len(ressenyes) > 0:
    ressenyes_tokenitzades, paraula_a_index = tokenitzar_ressenyes(ressenyes)
    ressenyes_padded = padding_sequencies(ressenyes_tokenitzades, max_len)

# Convertir a tensors
X = torch.tensor(ressenyes_padded, dtype=torch.long).to(device)
y = torch.tensor(etiquetes, dtype=torch.float).to(device)

# --- Pas 2: Dataset Personalitzat ---
class RessenyesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = RessenyesDataset(X, y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# --- Pas 3: Definició del Model ---
class NeuralNetwork(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers, output_size, dropout=0.5):
        super(NeuralNetwork, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # Per classificar entre 0 i 1

    def forward(self, x):
        x = self.embedding(x)  # Embeddings per a la seqüència d'índexs
        lstm_out, (hn, cn) = self.lstm(x)
        output = self.fc(hn[-1])  # Últim estat ocult
        return self.sigmoid(output)

    def __getitem__(self, idx):
        """
        Retorna un parell (X[idx], y[idx]) que correspon a la ressenya i la seva etiqueta.
        :param idx: Índex de l'element a retornar.
        """
        return self.X[idx], self.y[idx]


# Paràmetres del model
vocab_size = len(paraula_a_index)
embed_size = 128
hidden_size = 128
num_layers = 2
output_size = 1  # 1 classe (positiu o negatiu)
dropout = 0.5

model = NeuralNetwork(vocab_size, embed_size, hidden_size, num_layers, output_size, dropout).to(device)

# --- Pas 4: Entrenament del Model ---
# Paràmetres de l'entrenament
criteri = nn.BCELoss()  # Binary Cross Entropy
optimitzador = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimitzador.zero_grad()
        output = model(X_batch).to(device)
        loss = criteri(output.squeeze(), y_batch)
        loss.backward()
        optimitzador.step()
        total_loss += loss.item()
    print(f"Època {epoch+1}/{epochs}, Pèrdua: {total_loss/len(dataloader)}")

# --- Pas 5: Avaluació ---
def accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            output = model(X_batch).to(device)
            pred = (output.squeeze() > 0.5).float()
            correct += (pred == y_batch).sum().item()
            total += y_batch.size(0)
    return correct / total

print(f"Precisió del model: {accuracy(model, dataloader):.2f}")


# Mostra algunes prediccions del model
veure_prediccions(model, dataloader, paraula_a_index)


model_path = '/home/itibcn/Desktop/Torch/modelRessenyesCine.pth'
torch.save(model.state_dict(), model_path)



RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
